### Import Libraries

In [ ]:
import os
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

Now let’s read and import the images dataset that we are going to use to train a neural network model:

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale = 1.0/255)

train_generator = train_datagen.flow_from_directory("/kaggle/input/gender-recognition-200k-images-celeba/Dataset/Train",
                                                   batch_size=256,
                                                   class_mode='binary',
                                                   target_size=(64,64))

validation_generator = test_datagen.flow_from_directory("/kaggle/input/gender-recognition-200k-images-celeba/Dataset/Validation",
                                                       batch_size=256,
                                                       class_mode = 'binary',
                                                       target_size=(64,64))

Now we need to train and compile the neural network model for the task of Gender classification with Python:

In [ ]:
from keras.optimizers import Adam
model = tf.keras.models.Sequential([
    # 1st conv
    tf.keras.layers.Conv2D(96, (11,11), strides=(4,4), activation='relu', input_shape=(64,64,3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, strides=(2,2)),
    # 2nd conv
    tf.keras.layers.Conv2D(256, (11,11), strides=(1,1), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    # 3th conv
    tf.keras.layers.Conv2D(384, (3,3), strides=(1,1), activation='relu',padding="same"),
    tf.keras.layers.BatchNormalization(),
    # 4th conv
    tf.keras.layers.Conv2D(384, (3,3), strides=(1,1), activation='relu',padding="same"),
    tf.keras.layers.BatchNormalization(),
    # 5th conv
    tf.keras.layers.Conv2D(256, (3,3), strides=(1,1), activation='relu',padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, strides=(2,2)),
    # To Flatten layer
    tf.keras.layers.Flatten(),
    # To FC layer 1
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    #To FC layer 2
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])
model.compile(
    optimizer=Adam(lr=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
   )
hist = model.fit(train_generator,
                    validation_data=validation_generator,
                    steps_per_epoch=256,
                    validation_steps=256,
                    epochs=50)

Before testing this model let’s have a look at how the model performs in terms of accuracy and loss:

In [ ]:
import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

Now let’s test our neural network model in both the cases of male and female:

In [ ]:
import numpy as np

import keras.utils as image
# predicting images
path = "/kaggle/input/gender-recognition-200k-images-celeba/Dataset/Test/Female/160009.jpg"
img = image.load_img(path, target_size=(64, 64))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = model.predict(images, batch_size=1)
print(classes[0])
if classes[0]>0.5:
    print("is a man")
else:
    print( " is a female")
plt.imshow(img)